# Data and Data storage



In [1]:
# setup data needed for the notes
query_1 = "What are the benefits of renewable energy?" # gt is [0, 3]
query_2 = "How do solar panels impact the environment?" # gt is [1, 2]

org_documents =[
    {
        "title": "The Impact of Renewable Energy on the Economy",
        "content": "Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure."
    },
    {
        "title": "Understanding Solar Panels",
        "content": "Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock electrons free from atoms, generating a flow of electricity. Solar panels are a type of renewable energy technology that has been found to have a significant positive effect on the environment by reducing the reliance on fossil fuels."
    },
    {
        "title": "Pros and Cons of Solar Energy",
        "content": "While solar energy offers substantial environmental benefits, such as reducing carbon footprints and pollution, it also has downsides. The production of solar panels can lead to hazardous waste, and large solar farms require significant land, which can disrupt local ecosystems."
    },
    {
        "title":  "Renewable Energy and Its Effects",
        "content": "Renewable energy sources like wind, solar, and hydro power play a crucial role in combating climate change. They do not produce greenhouse gases during operation, making them essential for sustainable development. However, the initial setup and material sourcing for these technologies can still have environmental impacts."
    }
]

turns = [
    {
        "user": "What are the benefits of renewable energy?",
        "system": "I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.",
        "user_time": "2021-09-01T12:00:00Z",
        "system_time": "2021-09-01T12:00:01Z"
    },
    {
        "user": "How do solar panels impact the environment?",
        "system": "Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock electrons free from atoms, generating a flow of electricity. Solar panels are a type of renewable energy technology that has been found to have a significant positive effect on the environment by reducing the reliance on fossil fuels.",
        "user_time": "2021-09-01T12:00:02Z",
        "system_time": "2021-09-01T12:00:03Z"
    }
]

## Create Document and DialogTurn objects


In [2]:
# create Document objects
from lightrag.core.types import Document

# we will save the content to text and title in the meta_data
documents  = [Document(text=doc['content'], meta_data={'title': doc['title']}) for doc in org_documents]
print(documents)

[Document(id=41d383e7-266d-4d7b-9208-92455e963f58, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=1a6372a4-1b74-46e5-95e1-36833d33281c, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=3ebe149c-f36a-448d-918f-627140d48064, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=acee1c04-b5e9-4461-88fb-b868f87e6459, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combating climate ...', meta_dat

In [3]:
# create DialogTurn objects

from lightrag.core.types import DialogTurn, UserQuery, AssistantResponse

dialog_turns = [
    DialogTurn(
        user_query=UserQuery(query_str=turn["user"]),
        assistant_response=AssistantResponse(response_str=turn["system"]),
        user_query_timestamp=turn["user_time"],
        assistant_response_timestamp=turn["system_time"],
    )
    for turn in turns
]
print(dialog_turns)

[DialogTurn(id='0ec229c8-1396-43d6-999b-8816fb5337c6', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='What are the benefits of renewable energy?', metadata=None), assistant_response=AssistantResponse(response_str='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', metadata=None), user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='3d208368-2798-43fa-b645-6cf0c9f0daad', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='How do solar panels impact the environment?', metadata=None), assistant_response=AssistantResponse(response_str='Sola

## Demonstrating the data pipleine creation and application



In [4]:
# prepare the data pipeline

from lightrag.core.embedder import Embedder 
from lightrag.core.types import ModelClientType
from lightrag.components.data_process import DocumentSplitter, ToEmbeddings
from lightrag.core.component import Sequential


model_kwargs = {
    "model": "text-embedding-3-small",
    "dimensions": 256,
    "encoding_format": "float",
}

splitter_config = {
    "split_by": "word",
    "split_length": 50,
    "split_overlap": 10
}

splitter = DocumentSplitter(**splitter_config)
embedder = Embedder(model_client =ModelClientType.OPENAI(), model_kwargs=model_kwargs)
embedder_transformer = ToEmbeddings(embedder, batch_size=2)
data_transformer = Sequential(splitter, embedder_transformer)
print(data_transformer)

Sequential(
  (0): DocumentSplitter(split_by=word, split_length=50, split_overlap=10)
  (1): ToEmbeddings(
    batch_size=2
    (embedder): Embedder(
      model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
      (model_client): OpenAIClient()
    )
    (batch_embedder): BatchEmbedder(
      (embedder): Embedder(
        model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
        (model_client): OpenAIClient()
      )
    )
  )
)


In [5]:
# prepare mapping functions to map the data to Document object for the pipeline

from typing import Dict
# mapping function for org_documents
def map_to_document(doc: Dict) -> Document:
    return Document(text=doc['content'], meta_data={'title': doc['title']})

def map_dialogturn_to_document(turn: DialogTurn) -> Document:
    # it can be important to keep the original data's id
    return Document(id=turn.id, text=turn.user_query.query_str + ' ' + turn.assistant_response.response_str)

In [6]:
# apply data transformation to the dialog_turns

dialog_turns_as_documents = [map_dialogturn_to_document(turn) for turn in dialog_turns]
print(dialog_turns_as_documents)

# apply data transformation to the documents
output = data_transformer(dialog_turns_as_documents)
print(output)

[Document(id=0ec229c8-1396-43d6-999b-8816fb5337c6, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector=[], parent_doc_id=None, order=None, score=None), Document(id=3d208368-2798-43fa-b645-6cf0c9f0daad, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector=[], parent_doc_id=None, order=None, score=None)]


Batch embedding documents: 100%|██████████| 2/2 [00:00<00:00,  3.56it/s]
Adding embeddings to documents from batch: 2it [00:00, 36472.21it/s]

[Document(id=9f47b6e0-c28c-47df-ab70-c030891abc9f, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector='len: 256', parent_doc_id=0ec229c8-1396-43d6-999b-8816fb5337c6, order=0, score=None), Document(id=09a21701-fbfd-48de-83e7-1df721c92138, text='and installation sectors. The growth in renewable energy usage boosts local economies through increa...', meta_data=None, vector='len: 256', parent_doc_id=0ec229c8-1396-43d6-999b-8816fb5337c6, order=1, score=None), Document(id=bbe90fc0-7550-4447-b56e-e98cb6f96d3a, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector='len: 256', parent_doc_id=3d208368-2798-43fa-b645-6cf0c9f0daad, order=0, score=None), Document(id=affa5f3e-4f38-4427-a7e7-69a255653ead, text='has been found to have a significant positive effect on the environment by reducing the reliance on ...', meta_data=None, vector='

In [7]:
# apply data transformation to the documents
org_documents_as_documents = [map_to_document(doc) for doc in org_documents]

# apply data transformation to the documents
output = data_transformer(org_documents_as_documents)
print(output)

Batch embedding documents: 100%|██████████| 3/3 [00:00<00:00,  5.80it/s]
Adding embeddings to documents from batch: 3it [00:00, 9546.97it/s]

[Document(id=25379585-a84c-4ec4-9045-2dcd5aee0c2f, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector='len: 256', parent_doc_id=de5e15e3-d63d-4ebe-9e8f-24e6a63bb583, order=0, score=None), Document(id=f6b6351b-67b8-4689-957b-6f7fe3421605, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector='len: 256', parent_doc_id=3458c575-5d72-455f-ac46-a4e4ff7c4938, order=0, score=None), Document(id=98aad464-29c3-4d47-81bc-aff69ab3df49, text='positive effect on the environment by reducing the reliance on fossil fuels.', meta_data={'title': 'Understanding Solar Panels'}, vector='len: 256', parent_doc_id=3458c575-5d72-455f-ac46-a4e4ff7c4938, order=1, score=None), Document(id=6975f2a0-44c7-454e-9485-8983d4039e70, text='While solar energy offers substan

## Use LocalDB to help with the in-memory CRUD, and more importantly to keep track of data pipeline and its transformed data and with data storage and restore

In [8]:
# create a db for the dialog_turns
from lightrag.core.db import LocalDB

dialog_turn_db = LocalDB('dialog_turns')
print(dialog_turn_db)

dialog_turn_db.load(dialog_turns)
print(dialog_turn_db)

LocalDB(name='dialog_turns', items=[], transformed_items={}, transformer_setups={}, mapper_setups={})
LocalDB(name='dialog_turns', items=[DialogTurn(id='0ec229c8-1396-43d6-999b-8816fb5337c6', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='What are the benefits of renewable energy?', metadata=None), assistant_response=AssistantResponse(response_str='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', metadata=None), user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='3d208368-2798-43fa-b645-6cf0c9f0daad', user_id=None, session_id=None, order=None, user_query=U

In [9]:
# apply data transformation to the dialog_turn_db

key = "split_and_embed"
dialog_turn_db.transform(data_transformer, map_fn=map_dialogturn_to_document, key=key)
print(dialog_turn_db.transformed_items[key])
print(dialog_turn_db.transformer_setups[key])
print(dialog_turn_db.mapper_setups[key])

Batch embedding documents: 100%|██████████| 2/2 [00:00<00:00,  4.81it/s]
Adding embeddings to documents from batch: 2it [00:00, 14169.95it/s]

[Document(id=8a818f1a-a3b4-4e9b-b70e-6e3550849610, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector='len: 256', parent_doc_id=0ec229c8-1396-43d6-999b-8816fb5337c6, order=0, score=None), Document(id=7d841c77-2aba-4b42-94ee-82949821799f, text='and installation sectors. The growth in renewable energy usage boosts local economies through increa...', meta_data=None, vector='len: 256', parent_doc_id=0ec229c8-1396-43d6-999b-8816fb5337c6, order=1, score=None), Document(id=72cce418-94bf-470c-bf52-f3a9d9a463db, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector='len: 256', parent_doc_id=3d208368-2798-43fa-b645-6cf0c9f0daad, order=0, score=None), Document(id=5c1bbea0-456c-4a86-94c9-a4e9cfd800f0, text='has been found to have a significant positive effect on the environment by reducing the reliance on ...', meta_data=None, vector='

In [10]:
# save the state of the dialog_turn_db
dialog_turn_db.save_state('dialog_turn_db_state.pkl')

print(dialog_turn_db)

LocalDB(name='dialog_turns', items=[DialogTurn(id='0ec229c8-1396-43d6-999b-8816fb5337c6', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='What are the benefits of renewable energy?', metadata=None), assistant_response=AssistantResponse(response_str='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', metadata=None), user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='3d208368-2798-43fa-b645-6cf0c9f0daad', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='How do solar panels impact the environment?', metadata=None), assistant_response=A

In [11]:
# restore the state of the restored_dialog_turn_db
restored_dialog_turn_db = LocalDB.load_state('dialog_turn_db_state.pkl')
print(restored_dialog_turn_db)


LocalDB(name='dialog_turns', items=[DialogTurn(id='0ec229c8-1396-43d6-999b-8816fb5337c6', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='What are the benefits of renewable energy?', metadata=None), assistant_response=AssistantResponse(response_str='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', metadata=None), user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='3d208368-2798-43fa-b645-6cf0c9f0daad', user_id=None, session_id=None, order=None, user_query=UserQuery(query_str='How do solar panels impact the environment?', metadata=None), assistant_response=A

In [12]:
# check if the restored_dialog_turn_db is the same as the dialog_turn_db

print(str(dialog_turn_db.__dict__) == str(restored_dialog_turn_db.__dict__))

True


# CRUD operations

We will use a real user conversation by adding a generator

In [13]:
# prepare the generator for the dialog turns 

from lightrag.core import Generator

llm_kwargs = {
    "model": "gpt-3.5-turbo"
}

# we will use the default prompt, and using input_str and chat_history_str for the final prompt
generator = Generator(model_client = ModelClientType.OPENAI(), model_kwargs=llm_kwargs)
print(generator)



Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, 
  (prompt): Prompt(
    template: 
    {% if task_desc_str or output_format_str or tools_str or examples_str or chat_history_str or context_str or steps_str %}
    <SYS>
    {% endif %}
    {# task desc #}
    {% if task_desc_str %}
    {{task_desc_str}}
    {% endif %}
    {# output format #}
    {% if output_format_str %}
    <OUTPUT_FORMAT>
    {{output_format_str}}
    </OUTPUT_FORMAT>
    {% endif %}
    {# tools #}
    {% if tools_str %}
    <TOOLS>
    {{tools_str}}
    </TOOLS>
    {% endif %}
    {# example #}
    {% if examples_str %}
    <EXAMPLES>
    {{examples_str}}
    </EXAMPLES>
    {% endif %}
    {# chat history #}
    {% if chat_history_str %}
    <CHAT_HISTORY>
    {{chat_history_str}}
    </CHAT_HISTORY>
    {% endif %}
    {#contex#}
    {% if context_str %}
    <CONTEXT>
    {{context_str}}
    </CONTEXT>
    {% endif %}
    {# steps #}
    {% if steps_str %}
    <STEPS>
    {{steps_str}}
    </STEPS>
    {%

In [14]:
# lets see how the prompt will be if we pass the input_str and chat_history_str
input_str = "What are the benefits of renewable energy? Did I ask this before?" 

def format_chat_history_str(turns: list) -> str:
    chat_history_str = []
    for turn in turns:
        chat_history_str.append(turn.to_yaml()) # format as yaml
    # join with newline
    chat_history_str = '\n_________\n'.join(chat_history_str)
    return chat_history_str

chat_history_str = format_chat_history_str(dialog_turns)
print(generator.print_prompt(input_str=input_str, chat_history_str=chat_history_str))

Prompt:

<SYS>
<CHAT_HISTORY>
id: "0ec229c8-1396-43d6-999b-8816fb5337c6"
user_id: null
session_id: null
order: null
user_query: 
  metadata: null
  query_str: What are the benefits of renewable energy?
assistant_response: 
  metadata: null
  response_str: I can see you are interested in renewable energy. Renewable energy technologies
    not only help in reducing greenhouse gas emissions but also contribute significantly
    to the economy by creating jobs in the manufacturing and installation sectors. The
    growth in renewable energy usage boosts local economies through increased investment
    in technology and infrastructure
user_query_timestamp: "2021-09-01T12:00:00Z"
assistant_response_timestamp: "2021-09-01T12:00:01Z"
metadata: null
vector: null
_________
id: "3d208368-2798-43fa-b645-6cf0c9f0daad"
user_id: null
session_id: null
order: null
user_query: 
  metadata: null
  query_str: How do solar panels impact the environment?
assistant_response: 
  metadata: null
  response_str:

In [15]:
# as we have quite a bit of empty fields, lets exclude them 
from typing import List

input_str = "What are the benefits of renewable energy? Did I ask this before?" 

def format_chat_history_str(turns: List[DialogTurn]) -> str:
    chat_history_str = []
    for turn in turns:
        chat_history_str.append(
                    turn.to_yaml(
                        exclude=[
                            "id",
                            "user_id",
                            "session_id",
                            "user_query_timestamp",
                            "assistant_response_timestamp",
                            "order",
                            "metadata",
                            "vector",
                        ],
                    )
                )  
    chat_history_str = '\n_________\n'.join(chat_history_str)
    return chat_history_str

chat_history_str = format_chat_history_str(dialog_turn_db.items[0:1])
print(generator.print_prompt(input_str=input_str, chat_history_str=chat_history_str))

Prompt:

<SYS>
<CHAT_HISTORY>
user_query: 
  metadata: null
  query_str: What are the benefits of renewable energy?
assistant_response: 
  metadata: null
  response_str: I can see you are interested in renewable energy. Renewable energy technologies
    not only help in reducing greenhouse gas emissions but also contribute significantly
    to the economy by creating jobs in the manufacturing and installation sectors. The
    growth in renewable energy usage boosts local economies through increased investment
    in technology and infrastructure
</CHAT_HISTORY>
</SYS>
<User>
What are the benefits of renewable energy? Did I ask this before?
</User>
You:

None


In [16]:
print(dialog_turn_db.length, len(dialog_turn_db.transformed_items[key]))

2 4


In [17]:
prompt_kwargs = {"input_str": input_str, "chat_history_str": chat_history_str}

response = generator(prompt_kwargs=prompt_kwargs)
print(response)

GeneratorOutput(data='Yes, you asked about the benefits of renewable energy before. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', error=None, usage=None, raw_response='Yes, you asked about the benefits of renewable energy before. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.')


In [18]:
# create a turn from the last dialog

new_turn = DialogTurn(
    user_query=UserQuery(query_str=input_str),
    assistant_response=AssistantResponse(response_str=response.data),
)
dialog_turn_db.add(new_turn, apply_transformer=True)

Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00,  4.04it/s]
Adding embeddings to documents from batch: 1it [00:00, 5349.88it/s]


In [19]:
print(dialog_turn_db.length, len(dialog_turn_db.transformed_items[key]))

3 6


In [20]:
print(dialog_turn_db.transformed_items[key])

# we will find them by comparing the id of new_turn to the parent_doc_id in the transformed_items
new_turn_id = new_turn.id
print(new_turn_id)
for item in dialog_turn_db.transformed_items[key]:
    if item.parent_doc_id == new_turn_id:
        print(item)
        print(item.text)

[Document(id=64987b2b-b6c6-4eb4-9122-02448e3fd394, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector='len: 256', parent_doc_id=f2eddc77-4667-43f5-87e0-fd11f12958b3, order=0, score=None), Document(id=9a424d4c-4bd0-48ce-aba9-7a4f86892556, text='and installation sectors. The growth in renewable energy usage boosts local economies through increa...', meta_data=None, vector='len: 256', parent_doc_id=f2eddc77-4667-43f5-87e0-fd11f12958b3, order=1, score=None), Document(id=45efa517-8e52-4780-bdbd-2329ffa8d4b6, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector='len: 256', parent_doc_id=b2dbdf2f-f513-493d-aaa8-c77c98ac260f, order=0, score=None), Document(id=bc0ff7f6-27cc-4e24-8c3e-9435ed755e20, text='has been found to have a significant positive effect on the environment by reducing the reliance on ...', meta_data=None, vector='

### Loading and CRUD operations on documents

In [5]:
db.load(documents)
print(db)

LocalDB(name=None, items=[Document(id=6583698e-c676-4966-a0a3-91a0f3de3796, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=fab80c66-5da8-4639-9431-3f64f5829ab1, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=204e97ed-bd37-4d6f-b14e-e7518b4d1dc8, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=880bf5f9-7dde-4d4c-af74-83bdc82cbfba, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combati

In [6]:
# add and delete documents

new_document = Document(
    text="This is a new document", meta_data={'title': 'New Document'})

db.add(new_document)
print(db)

LocalDB(name=None, items=[Document(id=6583698e-c676-4966-a0a3-91a0f3de3796, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=fab80c66-5da8-4639-9431-3f64f5829ab1, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=204e97ed-bd37-4d6f-b14e-e7518b4d1dc8, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=880bf5f9-7dde-4d4c-af74-83bdc82cbfba, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combati

In [7]:
db.delete(indices=[-1])

In [8]:
len(db.documents)

AttributeError: 'LocalDB' object has no attribute 'documents'

In [ ]:
db.add(new_document, index=1)
print(db)
len(db.documents)

LocalDB(documents=[Document(id=b349a519-1139-42cb-9362-aa4a6a6919c4, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=49e80879-5b41-4f81-a236-ec512f2c60ad, text='This is a new document', meta_data={'title': 'New Document'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=12d12f6e-e269-4482-8b5b-7438a12d89b0, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=3ed98cb6-1ee2-4a96-b7fb-94bc5dcd3913, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, ord

5

In [ ]:
db.delete(indices=[1])
len(db.documents)

4

In [ ]:
db.delete(indices=[1])
len(db.documents)

3

In [ ]:
db

LocalDB(documents=[Document(id=b349a519-1139-42cb-9362-aa4a6a6919c4, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=3ed98cb6-1ee2-4a96-b7fb-94bc5dcd3913, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=7e13dac8-3a2f-4dcb-aeba-2941d2249e1d, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combating climate ...', meta_data={'title': 'Renewable Energy and Its Effects'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_documents={}, mapped_documents={}, transformer_setups={})

## data transformation (processing)

In [ ]:
from lightrag.core.base_data_class import DataClassFormatType
first_document = db.documents[0]
json_str = first_document.format_example_str(DataClassFormatType.EXAMPLE_JSON)
print(json_str)

{
    "text": "Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.",
    "meta_data": {
        "title": "The Impact of Renewable Energy on the Economy"
    },
    "vector": [],
    "id": "b349a519-1139-42cb-9362-aa4a6a6919c4",
    "order": "None",
    "score": "None",
    "parent_doc_id": "None",
    "estimated_num_tokens": 47
}


In [ ]:
yaml_str = first_document.format_example_str(DataClassFormatType.EXAMPLE_YAML)
print(yaml_str)

text: "Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure."
meta_data: title: The Impact of Renewable Energy on the Economy
vector: []
id: "b349a519-1139-42cb-9362-aa4a6a6919c4"
order: null
score: null
parent_doc_id: null
estimated_num_tokens: 47
